In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait

import time
from datetime import datetime
from bs4 import BeautifulSoup

import cx_Oracle

# 데이터베이스 연결 설정
connection = cx_Oracle.connect('kyb/1111@localhost:1521/xe')
cursor = connection.cursor()

# 웹드라이버 설정
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://map.naver.com/v5/search")
time.sleep(3)  # 페이지 로딩 대기

# 검색어 목록 가져오기
cursor.execute('SELECT id, search_name FROM zero') 
data_list = [(row[0], row[1]) for row in cursor.fetchall()]

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 현재 스크래핑된 HTML 요소 스냅샷 만드는 함수
def save_html_snapshot(page_str, file_prefix):
    current_date = datetime.now().strftime('%y%m%d_%H%M%S')
    file_path = f'{file_prefix}_{current_date}.html'
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(page_str)

# 해당 요소 찾을 때까지 10초 대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
        return wait
    except Exception as e:
        print(f"{code} 태그를 찾지 못하였습니다. {e}")
        return None

# 각 검색어에 대해 작업 수행
for id, search_name in data_list:
    try:
        # 검색창 찾기 및 검색어 입력
        search_box = time_wait(10, 'div.input_box > input.input_search')
        if search_box:
            search_box.clear()
            search_box.send_keys(search_name)
            search_box.send_keys(Keys.ENTER)
            time.sleep(3)  # 검색 결과 로딩 대기

            # 검색 결과 프레임으로 전환 및 스냅샷 저장
            switch_frame('searchIframe')
            save_html_snapshot(driver.page_source, f"search_result_{id}")

            # 리뷰 탭 클릭 및 프레임으로 전환
            switch_frame('entryIframe')
            reviews_tab_found = False
            for i in range(2, 7):
                try:
                    reviews_tab = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, f'//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[{i}]/span'))
                    )
                    if "리뷰" in reviews_tab.text:
                        reviews_tab.click()
                        reviews_tab_found = True
                        break
                except Exception as e:
                    print(f"리뷰 탭을 찾지 못하였습니다: {e}")
                    continue
            
            if not reviews_tab_found:
                print(f"리뷰 탭을 찾지 못하였습니다 for id: {id}")
                continue

            time.sleep(3)  # 리뷰 탭 로딩 대기
            save_html_snapshot(driver.page_source, f"review_{id}")

            # "내용 더보기" 버튼 클릭
            more_buttons = driver.find_elements(By.CLASS_NAME, 'rvCSr')
            for btn in more_buttons:
                try:
                    btn.click()
                    time.sleep(1)  # 버튼 클릭 후 로딩 대기
                except Exception as e:
                    print(f"Failed to click '내용 더보기' for ID {id}: {e}")

            # 업데이트된 페이지 소스 가져오기
            updated_page_source = driver.page_source
            save_html_snapshot(updated_page_source, f"updated_review_{id}")

            # 리뷰 텍스트 추출 및 데이터베이스 저장
            soup = BeautifulSoup(updated_page_source, 'html.parser')
            reviews = []
            review_elements = soup.find_all("span", class_="zPfVt")
            if not review_elements:
                review_elements = soup.find_all("div", class_="zPfVt")
            if not review_elements:
                print(f"No review elements found for ID {id}")

            for review in review_elements:
                review_text = review.get_text(separator=' ').strip()
                if review_text:
                    reviews.append(review_text)
                if len(reviews) >= 10:
                    break  # 최대 10개의 리뷰 텍스트만 추출

            # 디버깅: 추출된 리뷰 텍스트 출력
            print(f"ID {id}: Extracted {len(reviews)} reviews")
            for i, review in enumerate(reviews):
                print(f"Review {i+1}: {review}")

            # 데이터베이스에 리뷰 저장
            try:
                print("Binding variables for SQL query:")
                for i, review in enumerate(reviews):
                    print(f"review{i+1}: {review[:100] if review else None}")

                # 리뷰 텍스트 데이터베이스에 저장
                cursor.execute("""
                    INSERT INTO zero_re (id, re_num, review, review2, review3, review4, review5, review6, review7, review8, review9, review10)
                    VALUES (:id, zero_re_seq.NEXTVAL, :review, :review2, :review3, :review4, :review5, :review6, :review7, :review8, :review9, :review10)
                """, {
                    'id': id,
                    'review': reviews[0] if len(reviews) > 0 else None,
                    'review2': reviews[1] if len(reviews) > 1 else None,
                    'review3': reviews[2] if len(reviews) > 2 else None,
                    'review4': reviews[3] if len(reviews) > 3 else None,
                    'review5': reviews[4] if len(reviews) > 4 else None,
                    'review6': reviews[5] if len(reviews) > 5 else None,
                    'review7': reviews[6] if len(reviews) > 6 else None,
                    'review8': reviews[7] if len(reviews) > 7 else None,
                    'review9': reviews[8] if len(reviews) > 8 else None,
                    'review10': reviews[9] if len(reviews) > 9 else None
                })
                connection.commit()
                print(f"Successfully stored reviews for ID {id} in the database")  # 데이터베이스 저장 성공 로그
            except cx_Oracle.DatabaseError as db_e:
                error, = db_e.args
                print(f"Database error occurred while storing reviews for ID {id}: {error.message}")
                print(f"SQL Query: {cursor.statement}")
                print(f"Bound Variables: {cursor.bindvars()}")
            except Exception as e:
                print(f"Unexpected error occurred while storing reviews for ID {id}: {e}")
    except Exception as e:
        print(f"Error occurred while processing search_name '{search_name}' with id '{id}': {e}")

    # 검색어 처리 후 브라우저 새로 고침
    driver.get("https://map.naver.com/v5/search")
    time.sleep(3)

# 드라이버 종료 및 연결 종료
driver.quit()
cursor.close()
connection.close()


ID 1: Extracted 10 reviews
Review 1: 오랜만에 수세미 구매하고 갑니다❤️
Review 2: 디자인너무 이뻐요🌸
Review 3: 배러얼스 행사때마다 가는데
너무 좋은시간 보내고 왔어요!
아이들도 좋아하는 공간이에요
Review 4: 온라인에서 구매하려다가 가까운 곳에 위치한걸 보고 방문했어요~
매장이 작은데 솔이랑 바 종류가 알차게 여러가지 있어서 방문해서 이것저것 보기 좋았어요~
Review 5: 친환경, 비건 제품들이 다 모여있어요.
용기를 가져가면
필요한 양만큼 세제를 구입할 수 있어서 애용할 듯 합니다 ^^
샴푸바랑 목욕솔 구입해서  잘 쓰고 있답니다. 
오며가며 들러도 늘 친절하세요.
Review 6: 친환경 제품 ♡
가격이 싼편은 아니지만 환경을 생각해서 
또 건강한 삶을 위해서 가치있는 쇼핑을 했어요.
헤드가 교체되는 솔이 유용할듯하여 구매했고 두종류가 있는데 너무 강력한 솔은 후라이팬 🍳 코팅이 다 벗겨질수있다고하여
연한모질로 구매.
샤워그립이 참 좋네요. 오션타올만 쓰다 55브랜드에서 친환경 제품을 접하게된 이후부터 가급적이면 마, 수세미, 면제품을 선호하게되었어요.

실리콘 빨대 조립이 귀찮아서 👄 입에 닿는 느낌이 더 부드러운 유리빨대도 하나 구입.
수세미에서 씨앗 두개건짐.  이거 심으면 날까요?
Review 7: 다회용 지퍼백 사용해보고싶어서 실리콘지퍼백 샀어요.
가격이 좀 세지만 환경보호 차원에서 코엑스 행사중에 구입.
Review 8: 몇달만에 와서 수세미 사갖고 갑니다!
수세미 품질 최고
Review 9: 좋아요 좋아요
Review 10: 항상 믿고 구매합니다 ㅎ
Binding variables for SQL query:
review1: 오랜만에 수세미 구매하고 갑니다❤️
review2: 디자인너무 이뻐요🌸
review3: 배러얼스 행사때마다 가는데
너무 좋은시간 보내고 왔어요!
아이들도 좋아하는 공간이에요
review4: 온라인에서 구매하려다가 가까운 곳에 위치한걸 보고 방문했어요~
매장이 작은데 솔이